In [528]:
'''
   Copyright 2023 Spacebel s.a.

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
'''

'\n   Copyright 2023 Spacebel s.a.\n\n   Licensed under the Apache License, Version 2.0 (the "License");\n   you may not use this file except in compliance with the License.\n   You may obtain a copy of the License at\n\n       http://www.apache.org/licenses/LICENSE-2.0\n\n   Unless required by applicable law or agreed to in writing, software\n   distributed under the License is distributed on an "AS IS" BASIS,\n   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n   See the License for the specific language governing permissions and\n   limitations under the License.\n'

# Thesauri API

This notebook explains the use of the ESA Web Thesauri API.  Most examples use the Python requests module showing the invocations of the REST API.   Some examples use the `Skosmos-client` Python library that encapsulates the SKOSMOS REST API.

In [534]:
%pip install skosmos-client

import re
import folium
import json, requests, xml
import urllib.parse

from xml.dom import minidom
from xml.etree import ElementTree
from IPython.display import Markdown as md
from urllib.parse import urlparse, parse_qsl

HOST = "https://emc.spacebel.be/skosmos"
# HOST = "https://thesauri.eo.esa.int/skosmos"

API_URL = HOST + "/rest/v1/"
API_URL_DATA = API_URL + "thesaurus/data"
API_URL_SEARCH = API_URL + "thesaurus/search"

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [530]:
def curl_command( url: str, method: str = "GET" ) -> str:
    """
    Convert request URL to equivalent curl GET or POST command-line
    (bash shell).
    """
    c = "curl -X " + method
    res = urlparse(url)
    if "GET" in method:
        c = c + " -G " + res.scheme + "://" + res.netloc + res.path
    else:
        c = c + " " + res.scheme + "://" + res.netloc + res.path    \
              + " \\\n\t--header 'Content-Type: application/json'"  \
              + " \\\n\t--data-raw '{"
    
    lst = parse_qsl(res.query)
    
    first = True 
    for i in lst:
        # print(i[0])
        # add \ to end of previous line
        # correction 16/3: data-urlencode used.
        c = c + ' \\\n\t--data-urlencode "'+i[0]+'='+i[1]+'"'
        first = False

    return c

The Notebook can be used with a number of different endpoints.  Change the OSDD to be used for collection search by executing the notebook.

##  Get concept data

In [531]:

md("The RESTful API provided by the Thesaurus Browser is documented at " + API_URL + "/swagger.json")

The RESTful API provided by the Thesaurus Browser is documented at https://thesauri.eo.esa.int/skosmos/rest/v1//swagger.json

In [532]:
url = API_URL + "thesaurus/topConcepts"
print(url)

md("```shell\n" + curl_command(url) + "\n```\n")

https://thesauri.eo.esa.int/skosmos/rest/v1/thesaurus/topConcepts


```shell
curl -X GET -G https://thesauri.eo.esa.int/skosmos/rest/v1/thesaurus/topConcepts
```


In [533]:
response = requests.get( url )
# API returns a single top concept while each concept scheme has a different one ?

data = json.loads(response.text)
jstr = json.dumps(data, indent=3)
md("```json\n" + jstr + "\n```\n")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Platform data

**Example: 1.1**  
>   Get (Envisat) platform concept data in RDF/XML.

In [ ]:
concept_id = 'https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430'  # ENVISAT
# concept_id = 'https://earth.esa.int/concept/ae802b7d-b859-5e11-a052-97b267fa355c'  # ??

url = API_URL_DATA + "?uri=" + concept_id 
response = requests.get(url)
# response = requests.get(API_URL_DATA, params={ 'uri': concept_id, 'format': 'application/rdf+xml'} )

xmlstr = minidom.parseString(response.text).toprettyxml(indent='  ',newl='')
md("```shell\n" + curl_command(url) + "\n```\n" + "```xml\n" + xmlstr + "\n```\n")

```shell
curl -X GET -G https://emc.spacebel.be/skosmos/rest/v1/thesaurus/data \
	--data-urlencode "uri=https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
```
```xml
<?xml version="1.0" ?><rdf:RDF xmlns:ns0="https://schema.org/" xmlns:owl="http://www.w3.org/2002/07/owl#" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:skos="http://www.w3.org/2004/02/skos/core#" xmlns:sosa="http://www.w3.org/ns/sosa/">  

    <skos:Concept rdf:about="https://earth.esa.int/concept/5999338f-e606-5126-88b2-b0b12ec2c5ae">    
        <skos:prefLabel xml:lang="en">DORIS</skos:prefLabel>    
        <sosa:isHostedBy rdf:resource="https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"/>    
    </skos:Concept>  

    <skos:ConceptScheme rdf:about="https://earth.esa.int/concepts/concept_scheme/platforms">    
        <rdfs:label xml:lang="en">Platforms</rdfs:label>    
    </skos:ConceptScheme>  

    <skos:Concept rdf:about="https://earth.esa.int/concept/702277a0-8a84-5af3-a8f4-31d712ff17c3">    
        <skos:prefLabel xml:lang="en">AATSR</skos:prefLabel>    
        <sosa:isHostedBy rdf:resource="https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"/>    
    </skos:Concept>  

    <skos:Concept rdf:about="https://earth.esa.int/concept/ef96a1b4-a13f-5c0a-a67a-6589f9bdb833">    
        <skos:prefLabel xml:lang="en">ASAR</skos:prefLabel>    
        <sosa:isHostedBy rdf:resource="https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"/>    
    </skos:Concept>  

    <skos:Collection rdf:about="https://earth.esa.int/concepts/groups/heritage-missions">    
        <skos:prefLabel xml:lang="en">HM (Heritage Missions)</skos:prefLabel>    
        <skos:member rdf:resource="https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"/>    
    </skos:Collection>  

    <rdf:Description rdf:about="https://earth.esa.int/event/disposal/11ea961b-1d0b-5d6d-a55a-b58aed01d430">    
        <skos:prefLabel xml:lang="en">Disposal Envisat</skos:prefLabel>    
        <rdf:type rdf:resource="https://schema.org/Event"/>    
        <ns0:about rdf:resource="https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"/>    
    </rdf:Description>  

    <skos:Concept rdf:about="https://earth.esa.int/concept/c55d6941-023c-514c-a9c4-1693c053f861">    
        <rdfs:label xml:lang="en">Earth Observation Satellite</rdfs:label>    
        <skos:prefLabel xml:lang="en">Earth Observation Satellite</skos:prefLabel>    
        <skos:narrower rdf:resource="https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"/>    
    </skos:Concept>  

    <skos:Concept rdf:about="https://earth.esa.int/concept/cdb77e18-3620-56a8-be58-b7e89c25edb3">    
        <skos:prefLabel xml:lang="en">LRR</skos:prefLabel>    
        <sosa:isHostedBy rdf:resource="https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"/>    
    </skos:Concept>  

    <skos:Concept rdf:about="https://earth.esa.int/concept/003ed1d0-9f6c-59cb-a427-1fa4c4a92107">    
        <skos:prefLabel xml:lang="en">MERIS</skos:prefLabel>    
        <sosa:isHostedBy rdf:resource="https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"/>    
    </skos:Concept>  

    <skos:Concept rdf:about="https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430">    
        <sosa:hosts>      
            <skos:Concept rdf:about="https://earth.esa.int/concept/4bd77234-eabd-5691-8e0e-92a3f5305ae1">        
                <skos:prefLabel xml:lang="en">SCIAMACHY</skos:prefLabel>        
                <sosa:isHostedBy rdf:resource="https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"/>        
            </skos:Concept>      
        </sosa:hosts>    

        <sosa:hosts rdf:resource="https://earth.esa.int/concept/ef96a1b4-a13f-5c0a-a67a-6589f9bdb833"/>    
        <sosa:hosts>      
            <skos:Concept rdf:about="https://earth.esa.int/concept/2ebd3948-3f88-5944-8e3c-7e60099ec941">        
                <skos:prefLabel xml:lang="en">RA-2</skos:prefLabel>        
                <sosa:isHostedBy rdf:resource="https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"/>        
            </skos:Concept>      
        </sosa:hosts>    

        <sosa:hosts rdf:resource="https://earth.esa.int/concept/702277a0-8a84-5af3-a8f4-31d712ff17c3"/>    
        <sosa:hosts>      
            <skos:Concept rdf:about="https://earth.esa.int/concept/90c5a05d-6f00-5187-90b5-2963273124ed">        
                <skos:prefLabel xml:lang="en">MWR</skos:prefLabel>        
                <sosa:isHostedBy rdf:resource="https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"/>        
            </skos:Concept>      
        </sosa:hosts>    

        <sosa:hosts rdf:resource="https://earth.esa.int/concept/5999338f-e606-5126-88b2-b0b12ec2c5ae"/>    
        <sosa:hosts>      
            <skos:Concept rdf:about="https://earth.esa.int/concept/722d6f1d-b0d3-59be-b097-76efdb148ff9">        
                <skos:prefLabel xml:lang="en">MIPAS</skos:prefLabel>        
                <sosa:isHostedBy rdf:resource="https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"/>        
            </skos:Concept>      
        </sosa:hosts>    

        <sosa:hosts>      
            <skos:Concept rdf:about="https://earth.esa.int/concept/84156e47-190f-5056-a6c7-29b75855f8ca">        
                <skos:prefLabel xml:lang="en">GOMOS</skos:prefLabel>        
                <sosa:isHostedBy rdf:resource="https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"/>        
            </skos:Concept>      
        </sosa:hosts>    

        <sosa:hosts rdf:resource="https://earth.esa.int/concept/cdb77e18-3620-56a8-be58-b7e89c25edb3"/>    
        <sosa:hosts rdf:resource="https://earth.esa.int/concept/003ed1d0-9f6c-59cb-a427-1fa4c4a92107"/>    
        <ns0:event>      
            <ns0:Event rdf:about="https://earth.esa.int/event/launch/11ea961b-1d0b-5d6d-a55a-b58aed01d430">        
                <skos:prefLabel xml:lang="en">Launch Envisat</skos:prefLabel>        
                <ns0:about rdf:resource="https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"/>        
            </ns0:Event>      
        </ns0:event>    

        <ns0:event rdf:resource="https://earth.esa.int/event/disposal/11ea961b-1d0b-5d6d-a55a-b58aed01d430"/>    
        <skos:prefLabel xml:lang="en">Envisat</skos:prefLabel>    
        <skos:broader rdf:resource="https://earth.esa.int/concept/c55d6941-023c-514c-a9c4-1693c053f861"/>    
        <skos:notation xml:lang="en">EN1</skos:notation>    
        <owl:sameAs rdf:resource="http://dbpedia.org/resource/Envisat"/>    
        <owl:sameAs rdf:resource="http://www.wikidata.org/entity/Q49692"/>    
        <owl:sameAs rdf:resource="http://yago-knowledge.org/resource/Envisat"/>    
        <skos:exactMatch rdf:resource="https://gcmd.earthdata.nasa.gov/kms/concept/a1498dff-002d-4d67-9091-16822c608221"/>    
        <rdfs:seeAlso rdf:resource="https://earth.esa.int/eogateway/missions/envisat"/>    
        <skos:altLabel xml:lang="en">ENVISAT</skos:altLabel>    
        <skos:definition xml:lang="en">Envisat was ESA's successor to ERS. Envisat was launched in 2002 with 10 instruments aboard and at eight tons is the largest civilian Earth observation mission. More advanced imaging radar, radar altimeter and temperature-measuring radiometer instruments extend ERS data sets. This was supplemented by new instruments including a medium-resolution spectrometer sensitive to both land features and ocean colour. Envisat also carried two atmospheric sensors monitoring trace gases. The Envisat mission ended on 08 April 2012, following the unexpected loss of contact with the satellite.</skos:definition>    
        <skos:inScheme rdf:resource="https://earth.esa.int/concepts/concept_scheme/platforms"/>    
        <rdfs:label xml:lang="en">Envisat</rdfs:label>    
    </skos:Concept>  

    <rdf:Description rdf:about="http://www.w3.org/ns/sosa/hosts">    
        <rdfs:label xml:lang="en">hosts</rdfs:label>    
    </rdf:Description>  

    <rdf:Description rdf:about="https://schema.org/event">    
        <rdfs:label xml:lang="en">significant event</rdfs:label>    
    </rdf:Description>  

</rdf:RDF>
```


A platform concept refers to its instruments with the `sosa:hosts` property.

In [ ]:
# Some instruments may be referenced with rdf:resource
root = ElementTree.fromstring(response.text)
ns = {'skos': 'http://www.w3.org/2004/02/skos/core#', 
      'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
      'sosa': 'http://www.w3.org/ns/sosa/'}
for el in root.findall('skos:Concept[@rdf:about="' + concept_id + '"]/sosa:hosts[@rdf:resource]', ns):
      print (el.attrib.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource'))

https://earth.esa.int/concept/ef96a1b4-a13f-5c0a-a67a-6589f9bdb833
https://earth.esa.int/concept/702277a0-8a84-5af3-a8f4-31d712ff17c3
https://earth.esa.int/concept/5999338f-e606-5126-88b2-b0b12ec2c5ae
https://earth.esa.int/concept/cdb77e18-3620-56a8-be58-b7e89c25edb3
https://earth.esa.int/concept/003ed1d0-9f6c-59cb-a427-1fa4c4a92107


In [ ]:
# Other instruments may be embedded directly
for el in root.findall('./skos:Concept[@rdf:about="' + concept_id + '"]/sosa:hosts/skos:Concept', ns):
      # print ("attrib", el.attrib['rdf:about'])
      name = el.find('skos:prefLabel' , ns)
      # print(name.text)
      print (el.attrib.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about'), "(", name.text, ")")

https://earth.esa.int/concept/4bd77234-eabd-5691-8e0e-92a3f5305ae1 ( SCIAMACHY )
https://earth.esa.int/concept/2ebd3948-3f88-5944-8e3c-7e60099ec941 ( RA-2 )
https://earth.esa.int/concept/90c5a05d-6f00-5187-90b5-2963273124ed ( MWR )
https://earth.esa.int/concept/722d6f1d-b0d3-59be-b097-76efdb148ff9 ( MIPAS )
https://earth.esa.int/concept/84156e47-190f-5056-a6c7-29b75855f8ca ( GOMOS )


**Example: 1.2**  
>  Get (Envisat) platform concept data in JSON.

In [ ]:
url = API_URL_DATA + "?uri=" + concept_id + "&format=application/json"
response = requests.get(url)
# response = requests.get(API_URL_DATA, params={ 'uri': concept_id , 'format': 'application/json'} )

jstr = json.dumps(json.loads(response.text), indent=3)
md("```shell\n" + curl_command(url) + "\n```\n" + "```json\n" + jstr + "\n```\n")

```shell
curl -X GET -G https://emc.spacebel.be/skosmos/rest/v1/thesaurus/data \
	--data-urlencode "uri=https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430" \
	--data-urlencode "format=application/json"
```
```json
{
   "@context": {
      "skos": "http://www.w3.org/2004/02/skos/core#",
      "isothes": "http://purl.org/iso25964/skos-thes#",
      "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
      "owl": "http://www.w3.org/2002/07/owl#",
      "dct": "http://purl.org/dc/terms/",
      "dc11": "http://purl.org/dc/elements/1.1/",
      "uri": "@id",
      "type": "@type",
      "lang": "@language",
      "value": "@value",
      "graph": "@graph",
      "label": "rdfs:label",
      "prefLabel": "skos:prefLabel",
      "altLabel": "skos:altLabel",
      "hiddenLabel": "skos:hiddenLabel",
      "broader": "skos:broader",
      "narrower": "skos:narrower",
      "related": "skos:related",
      "inScheme": "skos:inScheme",
      "exactMatch": "skos:exactMatch",
      "closeMatch": "skos:closeMatch",
      "broadMatch": "skos:broadMatch",
      "narrowMatch": "skos:narrowMatch",
      "relatedMatch": "skos:relatedMatch"
   },
   "graph": [
      {
         "uri": "http://www.w3.org/ns/sosa/hosts",
         "label": {
            "lang": "en",
            "value": "hosts"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/003ed1d0-9f6c-59cb-a427-1fa4c4a92107",
         "type": "skos:Concept",
         "prefLabel": {
            "lang": "en",
            "value": "MERIS"
         },
         "http://www.w3.org/ns/sosa/isHostedBy": {
            "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430",
         "type": "skos:Concept",
         "label": {
            "lang": "en",
            "value": "Envisat"
         },
         "rdfs:seeAlso": {
            "uri": "https://earth.esa.int/eogateway/missions/envisat"
         },
         "owl:sameAs": [
            {
               "uri": "http://dbpedia.org/resource/Envisat"
            },
            {
               "uri": "http://www.wikidata.org/entity/Q49692"
            },
            {
               "uri": "http://yago-knowledge.org/resource/Envisat"
            }
         ],
         "altLabel": {
            "lang": "en",
            "value": "ENVISAT"
         },
         "broader": {
            "uri": "https://earth.esa.int/concept/c55d6941-023c-514c-a9c4-1693c053f861"
         },
         "skos:definition": {
            "lang": "en",
            "value": "Envisat was ESA's successor to ERS. Envisat was launched in 2002 with 10 instruments aboard and at eight tons is the largest civilian Earth observation mission. More advanced imaging radar, radar altimeter and temperature-measuring radiometer instruments extend ERS data sets. This was supplemented by new instruments including a medium-resolution spectrometer sensitive to both land features and ocean colour. Envisat also carried two atmospheric sensors monitoring trace gases. The Envisat mission ended on 08 April 2012, following the unexpected loss of contact with the satellite."
         },
         "exactMatch": {
            "uri": "https://gcmd.earthdata.nasa.gov/kms/concept/a1498dff-002d-4d67-9091-16822c608221"
         },
         "inScheme": {
            "uri": "https://earth.esa.int/concepts/concept_scheme/platforms"
         },
         "skos:notation": {
            "lang": "en",
            "value": "EN1"
         },
         "prefLabel": {
            "lang": "en",
            "value": "Envisat"
         },
         "http://www.w3.org/ns/sosa/hosts": [
            {
               "uri": "https://earth.esa.int/concept/4bd77234-eabd-5691-8e0e-92a3f5305ae1"
            },
            {
               "uri": "https://earth.esa.int/concept/ef96a1b4-a13f-5c0a-a67a-6589f9bdb833"
            },
            {
               "uri": "https://earth.esa.int/concept/2ebd3948-3f88-5944-8e3c-7e60099ec941"
            },
            {
               "uri": "https://earth.esa.int/concept/702277a0-8a84-5af3-a8f4-31d712ff17c3"
            },
            {
               "uri": "https://earth.esa.int/concept/90c5a05d-6f00-5187-90b5-2963273124ed"
            },
            {
               "uri": "https://earth.esa.int/concept/5999338f-e606-5126-88b2-b0b12ec2c5ae"
            },
            {
               "uri": "https://earth.esa.int/concept/722d6f1d-b0d3-59be-b097-76efdb148ff9"
            },
            {
               "uri": "https://earth.esa.int/concept/84156e47-190f-5056-a6c7-29b75855f8ca"
            },
            {
               "uri": "https://earth.esa.int/concept/cdb77e18-3620-56a8-be58-b7e89c25edb3"
            },
            {
               "uri": "https://earth.esa.int/concept/003ed1d0-9f6c-59cb-a427-1fa4c4a92107"
            }
         ],
         "https://schema.org/event": [
            {
               "uri": "https://earth.esa.int/event/launch/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
            },
            {
               "uri": "https://earth.esa.int/event/disposal/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
            }
         ]
      },
      {
         "uri": "https://earth.esa.int/concept/2ebd3948-3f88-5944-8e3c-7e60099ec941",
         "type": "skos:Concept",
         "prefLabel": {
            "lang": "en",
            "value": "RA-2"
         },
         "http://www.w3.org/ns/sosa/isHostedBy": {
            "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/4bd77234-eabd-5691-8e0e-92a3f5305ae1",
         "type": "skos:Concept",
         "prefLabel": {
            "lang": "en",
            "value": "SCIAMACHY"
         },
         "http://www.w3.org/ns/sosa/isHostedBy": {
            "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/5999338f-e606-5126-88b2-b0b12ec2c5ae",
         "type": "skos:Concept",
         "prefLabel": {
            "lang": "en",
            "value": "DORIS"
         },
         "http://www.w3.org/ns/sosa/isHostedBy": {
            "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/702277a0-8a84-5af3-a8f4-31d712ff17c3",
         "type": "skos:Concept",
         "prefLabel": {
            "lang": "en",
            "value": "AATSR"
         },
         "http://www.w3.org/ns/sosa/isHostedBy": {
            "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/722d6f1d-b0d3-59be-b097-76efdb148ff9",
         "type": "skos:Concept",
         "prefLabel": {
            "lang": "en",
            "value": "MIPAS"
         },
         "http://www.w3.org/ns/sosa/isHostedBy": {
            "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/84156e47-190f-5056-a6c7-29b75855f8ca",
         "type": "skos:Concept",
         "prefLabel": {
            "lang": "en",
            "value": "GOMOS"
         },
         "http://www.w3.org/ns/sosa/isHostedBy": {
            "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/90c5a05d-6f00-5187-90b5-2963273124ed",
         "type": "skos:Concept",
         "prefLabel": {
            "lang": "en",
            "value": "MWR"
         },
         "http://www.w3.org/ns/sosa/isHostedBy": {
            "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/c55d6941-023c-514c-a9c4-1693c053f861",
         "type": "skos:Concept",
         "label": {
            "lang": "en",
            "value": "Earth Observation Satellite"
         },
         "narrower": {
            "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
         },
         "prefLabel": {
            "lang": "en",
            "value": "Earth Observation Satellite"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/cdb77e18-3620-56a8-be58-b7e89c25edb3",
         "type": "skos:Concept",
         "prefLabel": {
            "lang": "en",
            "value": "LRR"
         },
         "http://www.w3.org/ns/sosa/isHostedBy": {
            "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/ef96a1b4-a13f-5c0a-a67a-6589f9bdb833",
         "type": "skos:Concept",
         "prefLabel": {
            "lang": "en",
            "value": "ASAR"
         },
         "http://www.w3.org/ns/sosa/isHostedBy": {
            "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
         }
      },
      {
         "uri": "https://earth.esa.int/concepts/concept_scheme/platforms",
         "type": "skos:ConceptScheme",
         "label": {
            "lang": "en",
            "value": "Platforms"
         }
      },
      {
         "uri": "https://earth.esa.int/concepts/groups/heritage-missions",
         "type": "skos:Collection",
         "skos:member": {
            "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
         },
         "prefLabel": {
            "lang": "en",
            "value": "HM (Heritage Missions)"
         }
      },
      {
         "uri": "https://earth.esa.int/event/disposal/11ea961b-1d0b-5d6d-a55a-b58aed01d430",
         "type": "https://schema.org/Event",
         "prefLabel": {
            "lang": "en",
            "value": "Disposal Envisat"
         },
         "https://schema.org/about": {
            "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
         }
      },
      {
         "uri": "https://earth.esa.int/event/launch/11ea961b-1d0b-5d6d-a55a-b58aed01d430",
         "type": "https://schema.org/Event",
         "prefLabel": {
            "lang": "en",
            "value": "Launch Envisat"
         },
         "https://schema.org/about": {
            "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
         }
      },
      {
         "uri": "https://schema.org/event",
         "label": {
            "lang": "en",
            "value": "significant event"
         }
      }
   ]
}
```


###  Instrument data

**Example: 2.1**  
>   Get instrument concept data in RDF/XML.

In [ ]:
concept_id = 'https://earth.esa.int/concept/003ed1d0-9f6c-59cb-a427-1fa4c4a92107'  # MERIS

url = API_URL_DATA + "?uri=" + concept_id 
response = requests.get(url)
# response = requests.get(API_URL_DATA, params={ 'uri': concept_id, 'format': 'application/rdf+xml'} )

xmlstr = minidom.parseString(response.text).toprettyxml(indent='  ',newl='')
md("```shell\n" + curl_command(url) + "\n```\n" + "```xml\n" + xmlstr + "\n```\n")

```shell
curl -X GET -G https://emc.spacebel.be/skosmos/rest/v1/thesaurus/data \
	--data-urlencode "uri=https://earth.esa.int/concept/003ed1d0-9f6c-59cb-a427-1fa4c4a92107"
```
```xml
<?xml version="1.0" ?><rdf:RDF xmlns:owl="http://www.w3.org/2002/07/owl#" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:skos="http://www.w3.org/2004/02/skos/core#" xmlns:sosa="http://www.w3.org/ns/sosa/">  

    <skos:Concept rdf:about="https://earth.esa.int/concept/66ab957a-0434-5953-8b33-be95150a9ecb">    
        <skos:prefLabel xml:lang="en">Imaging Spectrometers/Radiometers</skos:prefLabel>    
        <skos:narrower rdf:resource="https://earth.esa.int/concept/003ed1d0-9f6c-59cb-a427-1fa4c4a92107"/>    
    </skos:Concept>  

    <skos:ConceptScheme rdf:about="https://earth.esa.int/concepts/concept_scheme/instruments">    
        <rdfs:label xml:lang="en">Instruments</rdfs:label>    
    </skos:ConceptScheme>  

    <rdf:Description rdf:about="http://www.w3.org/ns/sosa/isHostedBy">    
        <rdfs:label xml:lang="en">is hosted by</rdfs:label>    
    </rdf:Description>  

    <skos:Concept rdf:about="https://earth.esa.int/concept/003ed1d0-9f6c-59cb-a427-1fa4c4a92107">    
        <skos:exactMatch rdf:resource="https://gcmd.earthdata.nasa.gov/kms/concept/fc3f3ca6-1578-47b2-9d99-bc35ca85e0ba"/>    
        <skos:prefLabel xml:lang="en">MERIS</skos:prefLabel>    
        <skos:inScheme rdf:resource="https://earth.esa.int/concepts/concept_scheme/instruments"/>    
        <owl:sameAs rdf:resource="http://dbpedia.org/resource/MERIS"/>    
        <owl:sameAs rdf:resource="http://yago-knowledge.org/resource/MERIS"/>    
        <owl:sameAs rdf:resource="http://www.wikidata.org/entity/Q6715420"/>    
        <skos:definition xml:lang="en">MERIS is a programmable, medium-spectral resolution, imaging spectrometer operating in the solar reflective spectral range. Fifteen spectral bands can be selected by ground command.

The instrument scans the Earth's surface by the so called &quot;push-broom&quot; method. Linear CCD arrays provide spatial sampling in the across-track direction, while the satellite's motion provides scanning in the along-track direction.</skos:definition>    
        <rdfs:seeAlso rdf:resource="https://earth.esa.int/web/guest/missions/esa-operational-eo-missions/envisat/instruments/meris"/>    
        <skos:broader rdf:resource="https://earth.esa.int/concept/66ab957a-0434-5953-8b33-be95150a9ecb"/>    
        <sosa:isHostedBy>      
            <skos:Concept rdf:about="https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430">        
                <skos:notation xml:lang="en">EN1</skos:notation>        
                <rdfs:label xml:lang="en">Envisat</rdfs:label>        
                <skos:prefLabel xml:lang="en">Envisat</skos:prefLabel>        
                <sosa:hosts rdf:resource="https://earth.esa.int/concept/003ed1d0-9f6c-59cb-a427-1fa4c4a92107"/>        
            </skos:Concept>      
        </sosa:isHostedBy>    

        <skos:altLabel xml:lang="en">MERIS</skos:altLabel>    
    </skos:Concept>  

</rdf:RDF>
```


The instrument concept refers to the platform on which it is available are available with the `sosa:hostedBy` property.

In [ ]:
# Some platforms may be referenced with rdf:resource
root = ElementTree.fromstring(response.text)
for el in root.findall('skos:Concept[@rdf:about="' + concept_id + '"]/sosa:isHostedBy[@rdf:resource]', ns):
      print (el.attrib.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource'))

In [ ]:
# Other platforms may be embedded directly
for el in root.findall('./skos:Concept[@rdf:about="' + concept_id + '"]/sosa:isHostedBy/skos:Concept', ns):
      name = el.find('skos:prefLabel' , ns)
      print (el.attrib.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about'), "(", name.text, ")")

https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430 ( Envisat )


**Example: 2.2**  
>  Get instrument concept data in JSON.

In [ ]:
url = API_URL_DATA + "?uri=" + concept_id + "&format=application/json"
response = requests.get(url)
# response = requests.get(API_URL_DATA, params={ 'uri': concept_id , 'format': 'application/json'} )

jstr = json.dumps(json.loads(response.text), indent=3)
md("```shell\n" + curl_command(url) + "\n```\n" + "```json\n" + jstr + "\n```\n")

```shell
curl -X GET -G https://emc.spacebel.be/skosmos/rest/v1/thesaurus/data \
	--data-urlencode "uri=https://earth.esa.int/concept/003ed1d0-9f6c-59cb-a427-1fa4c4a92107" \
	--data-urlencode "format=application/json"
```
```json
{
   "@context": {
      "skos": "http://www.w3.org/2004/02/skos/core#",
      "isothes": "http://purl.org/iso25964/skos-thes#",
      "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
      "owl": "http://www.w3.org/2002/07/owl#",
      "dct": "http://purl.org/dc/terms/",
      "dc11": "http://purl.org/dc/elements/1.1/",
      "uri": "@id",
      "type": "@type",
      "lang": "@language",
      "value": "@value",
      "graph": "@graph",
      "label": "rdfs:label",
      "prefLabel": "skos:prefLabel",
      "altLabel": "skos:altLabel",
      "hiddenLabel": "skos:hiddenLabel",
      "broader": "skos:broader",
      "narrower": "skos:narrower",
      "related": "skos:related",
      "inScheme": "skos:inScheme",
      "exactMatch": "skos:exactMatch",
      "closeMatch": "skos:closeMatch",
      "broadMatch": "skos:broadMatch",
      "narrowMatch": "skos:narrowMatch",
      "relatedMatch": "skos:relatedMatch"
   },
   "graph": [
      {
         "uri": "http://www.w3.org/ns/sosa/isHostedBy",
         "label": {
            "lang": "en",
            "value": "is hosted by"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/003ed1d0-9f6c-59cb-a427-1fa4c4a92107",
         "type": "skos:Concept",
         "rdfs:seeAlso": {
            "uri": "https://earth.esa.int/web/guest/missions/esa-operational-eo-missions/envisat/instruments/meris"
         },
         "owl:sameAs": [
            {
               "uri": "http://dbpedia.org/resource/MERIS"
            },
            {
               "uri": "http://yago-knowledge.org/resource/MERIS"
            },
            {
               "uri": "http://www.wikidata.org/entity/Q6715420"
            }
         ],
         "altLabel": {
            "lang": "en",
            "value": "MERIS"
         },
         "broader": {
            "uri": "https://earth.esa.int/concept/66ab957a-0434-5953-8b33-be95150a9ecb"
         },
         "skos:definition": {
            "lang": "en",
            "value": "MERIS is a programmable, medium-spectral resolution, imaging spectrometer operating in the solar reflective spectral range. Fifteen spectral bands can be selected by ground command.\n\nThe instrument scans the Earth's surface by the so called \"push-broom\" method. Linear CCD arrays provide spatial sampling in the across-track direction, while the satellite's motion provides scanning in the along-track direction."
         },
         "exactMatch": {
            "uri": "https://gcmd.earthdata.nasa.gov/kms/concept/fc3f3ca6-1578-47b2-9d99-bc35ca85e0ba"
         },
         "inScheme": {
            "uri": "https://earth.esa.int/concepts/concept_scheme/instruments"
         },
         "prefLabel": {
            "lang": "en",
            "value": "MERIS"
         },
         "http://www.w3.org/ns/sosa/isHostedBy": {
            "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430",
         "type": "skos:Concept",
         "label": {
            "lang": "en",
            "value": "Envisat"
         },
         "skos:notation": {
            "lang": "en",
            "value": "EN1"
         },
         "prefLabel": {
            "lang": "en",
            "value": "Envisat"
         },
         "http://www.w3.org/ns/sosa/hosts": {
            "uri": "https://earth.esa.int/concept/003ed1d0-9f6c-59cb-a427-1fa4c4a92107"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/66ab957a-0434-5953-8b33-be95150a9ecb",
         "type": "skos:Concept",
         "narrower": {
            "uri": "https://earth.esa.int/concept/003ed1d0-9f6c-59cb-a427-1fa4c4a92107"
         },
         "prefLabel": {
            "lang": "en",
            "value": "Imaging Spectrometers/Radiometers"
         }
      },
      {
         "uri": "https://earth.esa.int/concepts/concept_scheme/instruments",
         "type": "skos:ConceptScheme",
         "label": {
            "lang": "en",
            "value": "Instruments"
         }
      }
   ]
}
```


###  EarthTopic data

**Example: 3.1**  
>   Get Earth topic concept data in RDF/XML.

In [ ]:
concept_id = 'https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937'  # Agriculture

url = API_URL_DATA + "?uri=" + concept_id 
response = requests.get(url)
# response = requests.get(API_URL_DATA, params={ 'uri': concept_id, 'format': 'application/rdf+xml'} )

xmlstr = minidom.parseString(response.text).toprettyxml(indent='  ',newl='')
md("```shell\n" + curl_command(url) + "\n```\n" + "```xml\n" + xmlstr + "\n```\n")

```shell
curl -X GET -G https://emc.spacebel.be/skosmos/rest/v1/thesaurus/data \
	--data-urlencode "uri=https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"
```
```xml
<?xml version="1.0" ?><rdf:RDF xmlns:dc="http://purl.org/dc/terms/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:skos="http://www.w3.org/2004/02/skos/core#">  

    <skos:Concept rdf:about="https://earth.esa.int/concept/83c765cc-3e53-58d6-b64b-a9fa582254b7">    
        <skos:prefLabel xml:lang="en">Plant Phenology</skos:prefLabel>    
        <skos:broader rdf:resource="https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"/>    
    </skos:Concept>  

    <skos:Concept rdf:about="https://earth.esa.int/concept/acbc6bd6-80ac-51a1-bc4a-f6c2ec411908">    
        <skos:prefLabel xml:lang="en">Photon/Optical Detectors</skos:prefLabel>    
        <skos:related rdf:resource="https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"/>    
    </skos:Concept>  

    <skos:Concept rdf:about="https://earth.esa.int/concept/66ab957a-0434-5953-8b33-be95150a9ecb">    
        <skos:prefLabel xml:lang="en">Imaging Spectrometers/Radiometers</skos:prefLabel>    
        <skos:related rdf:resource="https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"/>    
    </skos:Concept>  

    <skos:ConceptScheme rdf:about="https://earth.esa.int/concepts/concept_scheme/earth-topics">    
        <rdfs:label xml:lang="en">Earth Topics</rdfs:label>    
    </skos:ConceptScheme>  

    <skos:Concept rdf:about="https://earth.esa.int/concept/0921b037-8274-5df4-992e-6a01f798c558">    
        <skos:prefLabel xml:lang="en">Reclamation, Revegetation and Restoration</skos:prefLabel>    
        <skos:broader rdf:resource="https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"/>    
    </skos:Concept>  

    <skos:Concept rdf:about="https://earth.esa.int/concept/385d0fc1-6909-5e8f-a4c4-39cb9bacedf5">    
        <skos:prefLabel xml:lang="en">Invasive Species</skos:prefLabel>    
        <skos:broader rdf:resource="https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"/>    
    </skos:Concept>  

    <skos:Concept rdf:about="https://earth.esa.int/concept/678defa4-9057-5dc5-9427-9bd674db644e">    
        <skos:prefLabel xml:lang="en">Fisheries</skos:prefLabel>    
        <skos:broader rdf:resource="https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"/>    
    </skos:Concept>  

    <skos:Concept rdf:about="https://earth.esa.int/concept/764a170a-e361-5ea8-838e-e56994de69b7">    
        <skos:prefLabel xml:lang="en">Cameras</skos:prefLabel>    
        <skos:related rdf:resource="https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"/>    
    </skos:Concept>  

    <skos:Concept rdf:about="https://earth.esa.int/concept/8f86e1ce-97c3-534d-b370-2692352992fe">    
        <skos:prefLabel xml:lang="en">Imaging Radars</skos:prefLabel>    
        <skos:related rdf:resource="https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"/>    
    </skos:Concept>  

    <skos:Concept rdf:about="https://earth.esa.int/concept/80694ace-7a2a-50b0-9c22-4a4b45c985a1">    
        <skos:prefLabel xml:lang="en">Forestry</skos:prefLabel>    
        <skos:broader rdf:resource="https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"/>    
    </skos:Concept>  

    <skos:Concept rdf:about="https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937">    
        <skos:narrower rdf:resource="https://earth.esa.int/concept/83c765cc-3e53-58d6-b64b-a9fa582254b7"/>    
        <skos:narrower rdf:resource="https://earth.esa.int/concept/80694ace-7a2a-50b0-9c22-4a4b45c985a1"/>    
        <skos:narrower>      
            <skos:Concept rdf:about="https://earth.esa.int/concept/edb8488d-1f9d-548c-abf4-1cdf2dfa61f6">        
                <skos:prefLabel xml:lang="en">Crops and Yields</skos:prefLabel>        
                <skos:broader rdf:resource="https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"/>        
            </skos:Concept>      
        </skos:narrower>    

        <skos:narrower rdf:resource="https://earth.esa.int/concept/385d0fc1-6909-5e8f-a4c4-39cb9bacedf5"/>    
        <skos:narrower>      
            <skos:Concept rdf:about="https://earth.esa.int/concept/afda7e88-62f8-5d7a-8d22-ae4878d90ad7">        
                <skos:prefLabel xml:lang="en">Aquaculture</skos:prefLabel>        
                <skos:broader rdf:resource="https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"/>        
            </skos:Concept>      
        </skos:narrower>    

        <skos:narrower>      
            <skos:Concept rdf:about="https://earth.esa.int/concept/988355eb-a637-53f2-beb3-20379a7653e4">        
                <skos:prefLabel xml:lang="en">Soils</skos:prefLabel>        
                <skos:broader rdf:resource="https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"/>        
            </skos:Concept>      
        </skos:narrower>    

        <skos:narrower>      
            <skos:Concept rdf:about="https://earth.esa.int/concept/51d548da-98f9-5022-b7cf-1d23e5d0ceaa">        
                <skos:prefLabel xml:lang="en">Diseases and Pests</skos:prefLabel>        
                <skos:broader rdf:resource="https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"/>        
            </skos:Concept>      
        </skos:narrower>    

        <skos:narrower rdf:resource="https://earth.esa.int/concept/0921b037-8274-5df4-992e-6a01f798c558"/>    
        <skos:narrower rdf:resource="https://earth.esa.int/concept/678defa4-9057-5dc5-9427-9bd674db644e"/>    
        <skos:narrower>      
            <skos:Concept rdf:about="https://earth.esa.int/concept/4adaa2a2-25ae-5909-b983-9899df9c52da">        
                <skos:prefLabel xml:lang="en">Agricultural Chemicals</skos:prefLabel>        
                <skos:broader rdf:resource="https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"/>        
            </skos:Concept>      
        </skos:narrower>    

        <skos:inScheme rdf:resource="https://earth.esa.int/concepts/concept_scheme/earth-topics"/>    
        <skos:closeMatch rdf:resource="https://www.eionet.europa.eu/gemet/en/concept/232"/>    
        <skos:broader>      
            <skos:Concept rdf:about="https://earth.esa.int/concept/5c476560-e0a3-554a-9187-187a90da1309">        
                <skos:prefLabel xml:lang="en">Earth Topic</skos:prefLabel>        
                <skos:narrower rdf:resource="https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"/>        
            </skos:Concept>      
        </skos:broader>    

        <skos:exactMatch rdf:resource="https://gcmd.earthdata.nasa.gov/kms/concept/a956d045-3b12-441c-8a18-fac7d33b2b4e"/>    
        <skos:related rdf:resource="https://earth.esa.int/concept/764a170a-e361-5ea8-838e-e56994de69b7"/>    
        <skos:related rdf:resource="https://earth.esa.int/concept/66ab957a-0434-5953-8b33-be95150a9ecb"/>    
        <skos:related rdf:resource="https://earth.esa.int/concept/8f86e1ce-97c3-534d-b370-2692352992fe"/>    
        <skos:related rdf:resource="https://earth.esa.int/concept/acbc6bd6-80ac-51a1-bc4a-f6c2ec411908"/>    
        <skos:prefLabel xml:lang="en">Agriculture</skos:prefLabel>    
        <dc:modified rdf:datatype="http://www.w3.org/2001/XMLSchema#date">2024-05-30</dc:modified>    
    </skos:Concept>  

</rdf:RDF>
```


**Example: 3.2**  
>   Get Earth topic concept data in JSON.

In [ ]:
url = API_URL_DATA + "?uri=" + concept_id + "&format=application/json"
response = requests.get(url)
# response = requests.get(API_URL_DATA, params={ 'uri': concept_id , 'format': 'application/json'} )

jstr = json.dumps(json.loads(response.text), indent=3)
md("```shell\n" + curl_command(url) + "\n```\n" + "```json\n" + jstr + "\n```\n")

```shell
curl -X GET -G https://emc.spacebel.be/skosmos/rest/v1/thesaurus/data \
	--data-urlencode "uri=https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937" \
	--data-urlencode "format=application/json"
```
```json
{
   "@context": {
      "skos": "http://www.w3.org/2004/02/skos/core#",
      "isothes": "http://purl.org/iso25964/skos-thes#",
      "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
      "owl": "http://www.w3.org/2002/07/owl#",
      "dct": "http://purl.org/dc/terms/",
      "dc11": "http://purl.org/dc/elements/1.1/",
      "uri": "@id",
      "type": "@type",
      "lang": "@language",
      "value": "@value",
      "graph": "@graph",
      "label": "rdfs:label",
      "prefLabel": "skos:prefLabel",
      "altLabel": "skos:altLabel",
      "hiddenLabel": "skos:hiddenLabel",
      "broader": "skos:broader",
      "narrower": "skos:narrower",
      "related": "skos:related",
      "inScheme": "skos:inScheme",
      "exactMatch": "skos:exactMatch",
      "closeMatch": "skos:closeMatch",
      "broadMatch": "skos:broadMatch",
      "narrowMatch": "skos:narrowMatch",
      "relatedMatch": "skos:relatedMatch"
   },
   "graph": [
      {
         "uri": "https://earth.esa.int/concept/0921b037-8274-5df4-992e-6a01f798c558",
         "type": "skos:Concept",
         "broader": {
            "uri": "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"
         },
         "prefLabel": {
            "lang": "en",
            "value": "Reclamation, Revegetation and Restoration"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/385d0fc1-6909-5e8f-a4c4-39cb9bacedf5",
         "type": "skos:Concept",
         "broader": {
            "uri": "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"
         },
         "prefLabel": {
            "lang": "en",
            "value": "Invasive Species"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/4adaa2a2-25ae-5909-b983-9899df9c52da",
         "type": "skos:Concept",
         "broader": {
            "uri": "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"
         },
         "prefLabel": {
            "lang": "en",
            "value": "Agricultural Chemicals"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/51d548da-98f9-5022-b7cf-1d23e5d0ceaa",
         "type": "skos:Concept",
         "broader": {
            "uri": "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"
         },
         "prefLabel": {
            "lang": "en",
            "value": "Diseases and Pests"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/5c476560-e0a3-554a-9187-187a90da1309",
         "type": "skos:Concept",
         "narrower": {
            "uri": "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"
         },
         "prefLabel": {
            "lang": "en",
            "value": "Earth Topic"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/66ab957a-0434-5953-8b33-be95150a9ecb",
         "type": "skos:Concept",
         "prefLabel": {
            "lang": "en",
            "value": "Imaging Spectrometers/Radiometers"
         },
         "related": {
            "uri": "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/678defa4-9057-5dc5-9427-9bd674db644e",
         "type": "skos:Concept",
         "broader": {
            "uri": "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"
         },
         "prefLabel": {
            "lang": "en",
            "value": "Fisheries"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/764a170a-e361-5ea8-838e-e56994de69b7",
         "type": "skos:Concept",
         "prefLabel": {
            "lang": "en",
            "value": "Cameras"
         },
         "related": {
            "uri": "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/80694ace-7a2a-50b0-9c22-4a4b45c985a1",
         "type": "skos:Concept",
         "broader": {
            "uri": "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"
         },
         "prefLabel": {
            "lang": "en",
            "value": "Forestry"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/83c765cc-3e53-58d6-b64b-a9fa582254b7",
         "type": "skos:Concept",
         "broader": {
            "uri": "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"
         },
         "prefLabel": {
            "lang": "en",
            "value": "Plant Phenology"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/8f86e1ce-97c3-534d-b370-2692352992fe",
         "type": "skos:Concept",
         "prefLabel": {
            "lang": "en",
            "value": "Imaging Radars"
         },
         "related": {
            "uri": "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/988355eb-a637-53f2-beb3-20379a7653e4",
         "type": "skos:Concept",
         "broader": {
            "uri": "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"
         },
         "prefLabel": {
            "lang": "en",
            "value": "Soils"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/acbc6bd6-80ac-51a1-bc4a-f6c2ec411908",
         "type": "skos:Concept",
         "prefLabel": {
            "lang": "en",
            "value": "Photon/Optical Detectors"
         },
         "related": {
            "uri": "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/afda7e88-62f8-5d7a-8d22-ae4878d90ad7",
         "type": "skos:Concept",
         "broader": {
            "uri": "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"
         },
         "prefLabel": {
            "lang": "en",
            "value": "Aquaculture"
         }
      },
      {
         "uri": "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937",
         "type": "skos:Concept",
         "dct:modified": {
            "type": "http://www.w3.org/2001/XMLSchema#date",
            "value": "2024-05-30"
         },
         "broader": {
            "uri": "https://earth.esa.int/concept/5c476560-e0a3-554a-9187-187a90da1309"
         },
         "closeMatch": {
            "uri": "https://www.eionet.europa.eu/gemet/en/concept/232"
         },
         "exactMatch": {
            "uri": "https://gcmd.earthdata.nasa.gov/kms/concept/a956d045-3b12-441c-8a18-fac7d33b2b4e"
         },
         "inScheme": {
            "uri": "https://earth.esa.int/concepts/concept_scheme/earth-topics"
         },
         "narrower": [
            {
               "uri": "https://earth.esa.int/concept/83c765cc-3e53-58d6-b64b-a9fa582254b7"
            },
            {
               "uri": "https://earth.esa.int/concept/80694ace-7a2a-50b0-9c22-4a4b45c985a1"
            },
            {
               "uri": "https://earth.esa.int/concept/edb8488d-1f9d-548c-abf4-1cdf2dfa61f6"
            },
            {
               "uri": "https://earth.esa.int/concept/385d0fc1-6909-5e8f-a4c4-39cb9bacedf5"
            },
            {
               "uri": "https://earth.esa.int/concept/afda7e88-62f8-5d7a-8d22-ae4878d90ad7"
            },
            {
               "uri": "https://earth.esa.int/concept/988355eb-a637-53f2-beb3-20379a7653e4"
            },
            {
               "uri": "https://earth.esa.int/concept/51d548da-98f9-5022-b7cf-1d23e5d0ceaa"
            },
            {
               "uri": "https://earth.esa.int/concept/0921b037-8274-5df4-992e-6a01f798c558"
            },
            {
               "uri": "https://earth.esa.int/concept/678defa4-9057-5dc5-9427-9bd674db644e"
            },
            {
               "uri": "https://earth.esa.int/concept/4adaa2a2-25ae-5909-b983-9899df9c52da"
            }
         ],
         "prefLabel": {
            "lang": "en",
            "value": "Agriculture"
         },
         "related": [
            {
               "uri": "https://earth.esa.int/concept/764a170a-e361-5ea8-838e-e56994de69b7"
            },
            {
               "uri": "https://earth.esa.int/concept/66ab957a-0434-5953-8b33-be95150a9ecb"
            },
            {
               "uri": "https://earth.esa.int/concept/8f86e1ce-97c3-534d-b370-2692352992fe"
            },
            {
               "uri": "https://earth.esa.int/concept/acbc6bd6-80ac-51a1-bc4a-f6c2ec411908"
            }
         ]
      },
      {
         "uri": "https://earth.esa.int/concept/edb8488d-1f9d-548c-abf4-1cdf2dfa61f6",
         "type": "skos:Concept",
         "broader": {
            "uri": "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"
         },
         "prefLabel": {
            "lang": "en",
            "value": "Crops and Yields"
         }
      },
      {
         "uri": "https://earth.esa.int/concepts/concept_scheme/earth-topics",
         "type": "skos:ConceptScheme",
         "label": {
            "lang": "en",
            "value": "Earth Topics"
         }
      }
   ]
}
```


### Other

The following interfaces apply to all concepts, thus platforms, instruments and Earth Topics

**Example: 4.1**  
>  List the labels for a requested concept.

In [ ]:
concept_id = "https://earth.esa.int/concept/3ea37733-9202-52b1-9bb0-592e87cdacbc" # IRS-P6

url = API_URL + "/label" + "?uri=" + concept_id 
response = requests.get(url)

jstr = json.dumps(json.loads(response.text), indent=3)
md("```shell\n" + curl_command(url) + "\n```\n" + "```json\n" + jstr + "\n```\n")

```shell
curl -X GET -G https://emc.spacebel.be/skosmos/rest/v1//label \
	--data-urlencode "uri=https://earth.esa.int/concept/3ea37733-9202-52b1-9bb0-592e87cdacbc"
```
```json
{
   "@context": {
      "skos": "http://www.w3.org/2004/02/skos/core#",
      "uri": "@id",
      "type": "@type",
      "prefLabel": "skos:prefLabel",
      "altLabel": "skos:altLabel",
      "hiddenLabel": "skos:hiddenLabel",
      "@language": null
   },
   "uri": "https://earth.esa.int/concept/3ea37733-9202-52b1-9bb0-592e87cdacbc",
   "prefLabel": "IRS-P6",
   "altLabel": [
      "ResourceSat-1"
   ]
}
```


## Use concept groups

**Example: 5.1**  
>  List the groups available in the vocabulary.

In [ ]:
url = API_URL + "thesaurus/groups" 
response = requests.get(url)

jstr = json.dumps(json.loads(response.text), indent=3)
md("```shell\n" + curl_command(url) + "\n```\n" + "```json\n" + jstr + "\n```\n")

```shell
curl -X GET -G https://emc.spacebel.be/skosmos/rest/v1/thesaurus/groups
```
```json
{
   "@context": {
      "skos": "http://www.w3.org/2004/02/skos/core#",
      "uri": "@id",
      "type": "@type",
      "onki": "http://schema.onki.fi/onki#",
      "prefLabel": "skos:prefLabel",
      "groups": "onki:hasGroup",
      "childGroups": {
         "@id": "skos:member",
         "@type": "@id"
      },
      "hasMembers": "onki:hasMembers",
      "@language": "en"
   },
   "uri": "",
   "groups": [
      {
         "uri": "https://earth.esa.int/concepts/groups/earth-explorers",
         "prefLabel": "EE (Earth Explorers)",
         "hasMembers": true
      },
      {
         "uri": "https://earth.esa.int/concepts/groups/earth-watch",
         "prefLabel": "EW (Earth Watch)",
         "hasMembers": true
      },
      {
         "uri": "https://earth.esa.int/concepts/groups/heritage-missions",
         "prefLabel": "HM (Heritage Missions)",
         "hasMembers": true
      },
      {
         "uri": "https://earth.esa.int/concepts/groups/mission-opportunities",
         "prefLabel": "MO (Mission Opportunity)",
         "hasMembers": true
      },
      {
         "uri": "https://earth.esa.int/concepts/groups/third-party-missions",
         "prefLabel": "TPM (Third Party Missions)",
         "hasMembers": true
      }
   ]
}
```


**Example: 5.2**  
>  Find group members.

In [ ]:
url = API_URL + "thesaurus/groupMembers" + "?uri=" + "https://earth.esa.int/concepts/groups/earth-explorers"
response = requests.get(url)

jstr = json.dumps(json.loads(response.text), indent=3)
md("```shell\n" + curl_command(url) + "\n```\n" + "```json\n" + jstr + "\n```\n")

```shell
curl -X GET -G https://emc.spacebel.be/skosmos/rest/v1/thesaurus/groupMembers \
	--data-urlencode "uri=https://earth.esa.int/concepts/groups/earth-explorers"
```
```json
{
   "@context": {
      "skos": "http://www.w3.org/2004/02/skos/core#",
      "uri": "@id",
      "type": "@type",
      "prefLabel": "skos:prefLabel",
      "members": "skos:member",
      "@language": "en"
   },
   "uri": "https://earth.esa.int/concepts/groups/earth-explorers",
   "members": [
      {
         "uri": "https://earth.esa.int/concept/01d0faf3-e82a-587a-86c2-2b8d8bd52185",
         "isSuper": false,
         "hasMembers": false,
         "type": [
            "skos:Concept"
         ],
         "prefLabel": "Aeolus",
         "notation": "AE1"
      },
      {
         "uri": "https://earth.esa.int/concept/a25a98f2-c03a-51b7-be4a-afab0863186c",
         "isSuper": false,
         "hasMembers": false,
         "type": [
            "skos:Concept"
         ],
         "prefLabel": "Biomass",
         "notation": "BIO"
      },
      {
         "uri": "https://earth.esa.int/concept/58fa3406-73c7-5e87-9701-6cade8ba2c2b",
         "isSuper": false,
         "hasMembers": false,
         "type": [
            "skos:Concept"
         ],
         "prefLabel": "CryoSat",
         "notation": "CR2"
      },
      {
         "uri": "https://earth.esa.int/concept/fc478009-7c76-54af-ba48-64c36705faef",
         "isSuper": false,
         "hasMembers": false,
         "type": [
            "skos:Concept"
         ],
         "prefLabel": "EarthCARE",
         "notation": "ECA"
      },
      {
         "uri": "https://earth.esa.int/concept/0d74ba05-9f2d-5dec-8e99-f756f7b06c56",
         "isSuper": false,
         "hasMembers": false,
         "type": [
            "skos:Concept"
         ],
         "prefLabel": "FLEX",
         "notation": "FL"
      },
      {
         "uri": "https://earth.esa.int/concept/07603ff0-3940-5c7a-bf47-98c4974e637b",
         "isSuper": false,
         "hasMembers": false,
         "type": [
            "skos:Concept"
         ],
         "prefLabel": "FORUM"
      },
      {
         "uri": "https://earth.esa.int/concept/1abfac39-23bf-561f-a765-76da42a79d44",
         "isSuper": false,
         "hasMembers": false,
         "type": [
            "skos:Concept"
         ],
         "prefLabel": "GOCE",
         "notation": "GO1"
      },
      {
         "uri": "https://earth.esa.int/concept/2fba3b73-669d-56c3-8bbf-212f5eb749c7",
         "isSuper": false,
         "hasMembers": false,
         "type": [
            "skos:Concept"
         ],
         "prefLabel": "HARMONY"
      },
      {
         "uri": "https://earth.esa.int/concept/7eb92e2a-7b12-56a7-be1a-a71c97778559",
         "isSuper": false,
         "hasMembers": false,
         "type": [
            "skos:Concept"
         ],
         "prefLabel": "SMOS",
         "notation": "SM1"
      },
      {
         "uri": "https://earth.esa.int/concept/c146a188-953f-5492-ab15-2e83c545d5cb",
         "isSuper": false,
         "hasMembers": false,
         "type": [
            "skos:Concept"
         ],
         "prefLabel": "Swarm",
         "notation": "SWA,SWB,SWC"
      }
   ]
}
```


## Search concepts

**Example: 6.1**  
>  Search concepts and collections by query term.

In [ ]:
searchTerm = 'agriculture'
# searchTerm = 'sentinel-*'

url = API_URL_SEARCH + "?query=" + searchTerm + "&maxhits=3"
response = requests.get(url)

jstr = json.dumps(json.loads(response.text), indent=3)
md("```shell\n" + curl_command(url) + "\n```\n" + "```json\n" + jstr + "\n```\n")

```shell
curl -X GET -G https://emc.spacebel.be/skosmos/rest/v1/thesaurus/search \
	--data-urlencode "query=agriculture" \
	--data-urlencode "maxhits=3"
```
```json
{
   "@context": {
      "skos": "http://www.w3.org/2004/02/skos/core#",
      "isothes": "http://purl.org/iso25964/skos-thes#",
      "onki": "http://schema.onki.fi/onki#",
      "uri": "@id",
      "type": "@type",
      "results": {
         "@id": "onki:results",
         "@container": "@list"
      },
      "prefLabel": "skos:prefLabel",
      "altLabel": "skos:altLabel",
      "hiddenLabel": "skos:hiddenLabel"
   },
   "uri": "",
   "results": [
      {
         "uri": "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937",
         "type": [
            "skos:Concept"
         ],
         "localname": "bbe13d71-bea9-55c2-9b65-a24043c35937",
         "prefLabel": "Agriculture",
         "lang": "en",
         "vocab": "thesaurus"
      }
   ]
}
```


**Example: 6.2**  
>  Search concepts by query term belonging to a group (e.g. Heritage Missions).

In [ ]:
searchTerm = 'e*'   #  Search for concepts starting with "e"

url = API_URL_SEARCH + "?query=" + searchTerm \
    + "&group=" + 'https://earth.esa.int/concepts/groups/heritage-missions' \
    + "&maxhits=5"
response = requests.get(url)

jstr = json.dumps(json.loads(response.text), indent=3)
md("```shell\n" + curl_command(url) + "\n```\n" + "```json\n" + jstr + "\n```\n")

```shell
curl -X GET -G https://emc.spacebel.be/skosmos/rest/v1/thesaurus/search \
	--data-urlencode "query=e*" \
	--data-urlencode "group=https://earth.esa.int/concepts/groups/heritage-missions" \
	--data-urlencode "maxhits=5"
```
```json
{
   "@context": {
      "skos": "http://www.w3.org/2004/02/skos/core#",
      "isothes": "http://purl.org/iso25964/skos-thes#",
      "onki": "http://schema.onki.fi/onki#",
      "uri": "@id",
      "type": "@type",
      "results": {
         "@id": "onki:results",
         "@container": "@list"
      },
      "prefLabel": "skos:prefLabel",
      "altLabel": "skos:altLabel",
      "hiddenLabel": "skos:hiddenLabel"
   },
   "uri": "",
   "results": [
      {
         "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430",
         "type": [
            "skos:Concept"
         ],
         "localname": "11ea961b-1d0b-5d6d-a55a-b58aed01d430",
         "prefLabel": "Envisat",
         "lang": "en",
         "notation": "EN1",
         "vocab": "thesaurus"
      },
      {
         "uri": "https://earth.esa.int/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430",
         "type": [
            "skos:Concept"
         ],
         "localname": "11ea961b-1d0b-5d6d-a55a-b58aed01d430",
         "prefLabel": "Envisat",
         "lang": "en",
         "notation": "EN1",
         "altLabel": "ENVISAT",
         "vocab": "thesaurus"
      },
      {
         "uri": "https://earth.esa.int/concept/41660a0e-b024-5f96-bac7-03d20444800b",
         "type": [
            "skos:Concept"
         ],
         "localname": "41660a0e-b024-5f96-bac7-03d20444800b",
         "prefLabel": "ERS-1",
         "lang": "en",
         "notation": "ER1",
         "vocab": "thesaurus"
      },
      {
         "uri": "https://earth.esa.int/concept/ccf4c4fa-e698-5c16-a074-f958ccabce5e",
         "type": [
            "skos:Concept"
         ],
         "localname": "ccf4c4fa-e698-5c16-a074-f958ccabce5e",
         "prefLabel": "ERS-2",
         "lang": "en",
         "notation": "ER2",
         "vocab": "thesaurus"
      }
   ]
}
```


## Advanced topics

### Using the Skosmos-client

In [ ]:
from skosmos_client import SkosmosClient

concept_id = "https://earth.esa.int/concept/bbe13d71-bea9-55c2-9b65-a24043c35937"

# create your own client
skosmos = SkosmosClient( api_base=  API_URL )

print()
print("* Finding the available vocabularies")
for vocab in skosmos.vocabularies(lang='en'):
        print("Vocabulary id: {:<16} title: {}".format(
            vocab['id'], vocab['title']))
        
print()
print("* Looking up information about a particular vocabulary")
ths = skosmos.get_vocabulary('thesaurus', lang='en')
for key, val in sorted(ths.items()):
        if key in ('@context', 'uri'):
            continue
        print("{:<20}: {}".format(key, val))

print()        
print("* Searching for concepts")
for result in skosmos.search("ERS*", vocabs="thesaurus", lang="en"):
        print(result)
        
print()
print("* Looking up the thematic groups of a vocabulary")
for group in skosmos.groups('thesaurus', lang='en'):
        print(group)
        
print()
print("* Performing operations on single concepts")
prams = skosmos.get_concept('thesaurus', concept_id )
print("concept label in the default language:", prams.label())
print("concept label in English:", prams.label('en'))
print("Broader concepts:")
for bc in prams.broader('en'):
        print(bc)
print("Transitive broader concepts:")
for btc in prams.broaderTransitive('en'):
        print(btc)

print("Narrower concepts:")
hanko = skosmos.get_concept('thesaurus', concept_id )
for nc in hanko.narrower():
        print(nc)
print("Related concepts:")
for rc in prams.related('en'):
        print(rc)



* Finding the available vocabularies
Vocabulary id: thesaurus        title: ESA Thesaurus

* Looking up information about a particular vocabulary
conceptschemes      : [{'label': 'Earth Topics', 'title': 'ESA Thesaurus', 'uri': 'https://earth.esa.int/concepts/concept_scheme/earth-topics', 'type': 'skos:ConceptScheme'}, {'label': 'Instruments', 'title': 'ESA Thesaurus', 'uri': 'https://earth.esa.int/concepts/concept_scheme/instruments', 'type': 'skos:ConceptScheme'}, {'label': 'Platforms', 'uri': 'https://earth.esa.int/concepts/concept_scheme/platforms', 'type': 'skos:ConceptScheme'}]
defaultLanguage     : en
id                  : thesaurus
languages           : ['en']
marcSource          : False
title               : ESA Thesaurus
type                : [{'uri': 'http://publications.europa.eu/resource/authority/dataset-type/THESAURUS', 'prefLabel': 'Thesaurus'}]

* Searching for concepts
{'uri': 'https://earth.esa.int/concept/6caeef70-6328-53c7-a13f-b0da23076e9f', 'type': ['skos:Concep

<a name='Further-Reading'></a>  
## Further Reading

| **ID**  | **Title** | 
| -------- | --------- | 
| `RD1` <a name="RD1"></a> | [SKOS Simple Knowledge Organization System Primer, W3C Working Group Note, 18 August 2009.](https://www.w3.org/TR/skos-primer/) | 
| `RD2` <a name="RD2"></a> | [SKOS Simple Knowledge Organization System Reference, W3C Recommendation](https://www.w3.org/TR/skos-reference/) | 
| `RD3` <a name="RD3"></a> | [Skosmos REST API v1](https://api.finto.fi/doc/) | 
| `RD4` <a name="RD4"></a> |  [Skosmos-client Python client library](https://github.com/NatLibFi/Skosmos-client) |